In [ ]:
import pandas as pd
data = pd.read_csv("rainfall_data.csv")
data=data.iloc[:,3:]
# calculate 95th percentile for each column
percentiles = data.quantile(0.95)

# filter rows above 95th percentile for each column
filtered_data = data[(data > percentiles).any(axis=1)]




print(filtered_data)


In [ ]:
percentiles

In [ ]:
data = pd.read_csv("rainfall_data.csv")
# label extreme rows in original dataframe
data['rainfall_class'] = 'normal'
data.loc[data.index.isin(filtered_data.index), 'rainfall_class'] = 'extreme'


In [ ]:
import matplotlib.pyplot as plt

# Count the number of extreme and normal rainfall values
extreme_count = len(data[data['rainfall_class'] == 'extreme'])
normal_count = len(data[data['rainfall_class'] == 'normal'])

# Plot the counts as a bar chart
plt.bar(['Normal', 'Extreme'], [normal_count, extreme_count], color=['blue', 'red'])

# Add labels and title
plt.xlabel('Rainfall Class')
plt.ylabel('Count')
plt.title('Rainfall Data')

# Show the plot
plt.show()


In [ ]:
import numpy as np
import copy
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras import backend as K
import os
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import seed
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc



In [ ]:
extreme_data=data[data["rainfall_class"]=="extreme"]
normal_data=data[data["rainfall_class"]=="normal"]

In [ ]:
def weighted_loss(sparsity_weight=0.01, sparsity_target=0.001):
    def loss(y_true, y_pred):
        mse_loss = K.mean(K.square(abs(y_true[:,2] - y_pred[:,2])), axis=-1)
        kl_divergence = sparsity_weight * K.sum(sparsity_target * K.log(abs(sparsity_target / K.mean(abs(y_pred[:, 2])))) 
                        + abs((1 - sparsity_target)) * K.log(abs((1 - sparsity_target) / (1 - K.mean(abs(y_pred[:, 2]))))))
        return mse_loss + kl_divergence
    return loss

In [ ]:
def find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size):
    best_reg_strength = None
    best_loss = float('inf')
    for reg_strength in reg_strengths:
        # Define the autoencoder architecture
        input_layer = Input(shape=(3,))
        encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=reg_strength))(input_layer)
        decoded = Dense(3, activation=None)(encoded)

        # Create the autoencoder
        autoencoder = Model(input_layer, decoded)

        # Compile the autoencoder with the weighted loss function
        #autoencoder.compile(optimizer='adam', loss="mse")
        autoencoder.compile(optimizer='adam', loss=weighted_loss())
        # Train the model
        history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
        #history = autoencoder.fit(train_data,epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

        # Calculate the validation loss
        val_loss = np.mean(history.history['val_loss'])

        # Update the best regularization strength and loss
        if val_loss < best_loss:
            best_reg_strength = reg_strength
            best_loss = val_loss

    print('Best regularization strength:', best_reg_strength)
    return best_reg_strength

In [ ]:

reg_strengths = [0.1, 1, 10, 100]
num_epochs = 3000
batch_size = 100
#train_data = normal_data[["windspeed_discretized","tpw_discretized"]].values.astype("float32")
train_data = normal_data[["windspeed","tpw","rainfall"]].values.astype("float32")

#best_reg_strength = find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size)

# Define the autoencoder architecture with the best regularization strength
input_layer = Input(shape=(3,),name='input')
encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=20),name='encoder')(input_layer)
decoded = Dense(3, activation=None, name='decoder')(encoded)

# Create the autoencoder
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder with the weighted loss function
#autoencoder.compile(optimizer='adam', loss="mse")
autoencoder.compile(optimizer='adam', loss=weighted_loss())

# Train the model with the majority class
history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)
#history = autoencoder.fit(train_data,epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)


In [ ]:
from tensorflow.keras.models import load_model

# Save the model to a file
autoencoder.save("my_autoencoder_model.h5")


In [ ]:
#test_data = data[["windspeed_discretized","tpw_discretized","rainfall_discretized"]].values
test_data = data[["windspeed","tpw","rainfall"]].values
autoencoder = load_model("my_autoencoder_model.h5", custom_objects={"loss": weighted_loss()})

predicted_data = autoencoder.predict(test_data)

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model

# Define the custom loss function

# Load your autoencoder model
autoencoder = load_model("my_autoencoder_model.h5", custom_objects={"loss": weighted_loss()})
#autoencoder = load_model("my_autoencoder_model.h5")
# Create a Keras model to get the output of the encoder and decoder layers
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer("encoder").output)
decoder = Model(inputs=autoencoder.get_layer("decoder").input, outputs=autoencoder.output)

# Get the encoded representation of the input data
encoded_X = encoder.predict(test_data)

# Reconstruct the input data from the encoded representation
reconstructed_X = decoder.predict(encoded_X)
#print(test_data)
print(reconstructed_X)
# Compute the MSE reconstruction loss for each sample
mse_loss = np.sum((abs(test_data - abs( reconstructed_X))), axis=1)
#mse_loss = abs(np.sum(test_data)-np.sum(reconstructed_X), axis =1)
data["mse_loss"]=mse_loss
# Print the MSE reconstruction loss for each sample
print(data["mse_loss"])
top10_indices = mse_loss.argsort()[::-1][:10]

# Print the MSE reconstruction loss for each of the top 10 samples
for i in top10_indices:
    print('MSE for sample', i+1, ':', mse_loss[i])
sample_idx = 2387  # 0-based index of sample 5
print('MSE loss for sample 2387:', mse_loss[sample_idx])
sample_idx = 2388  # 0-based index of sample 5
print('MSE loss for sample 2388:', mse_loss[sample_idx])

In [ ]:
import matplotlib.pyplot as plt

# Create a histogram of the MSE loss distribution
plt.hist(mse_loss, bins=50)
plt.xlabel("MSE Loss")
plt.ylabel("Frequency")
plt.title("MSE Loss Distribution")
plt.show()


In [ ]:
df_sorted = data.sort_values('mse_loss', ascending=False)

In [ ]:
df_sorted

In [ ]:
df_max = df_sorted.nlargest(129, 'mse_loss')
threshold=np.mean(df_max["mse_loss"])

In [ ]:
data['predicted_rainfall_class'] = np.where(data['mse_loss'] > threshold, 'extreme', 'normal')
data[data["mse_loss"]>threshold]

In [ ]:
wrong_predictions = data[data['predicted_rainfall_class'] != data['rainfall_class']]

percentage_of_error=(wrong_predictions.shape[0]/data.shape[0])*100

# Print the wrong predictions
print("wrong predictions = ",wrong_predictions.shape[0])
print("total data = ",data.shape[0])
print("percentage of error = ",percentage_of_error)

In [ ]:
data['actual_rainfall_binary'] = (data['predicted_rainfall_class'] == data['rainfall_class'])

# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(data['actual_rainfall_binary'], data['mse_loss'])
roc_auc = auc(fpr, tpr)
# Calculate TNR
tnr = 1 - fpr

# Calculate Youden's J statistic for each threshold
j_stat = tpr + tnr - 1

# Find the index of the threshold that maximizes J
best_threshold_idx = np.argmax(j_stat)

# Get the best threshold
best_threshold = thresholds[best_threshold_idx]

plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()


# Print the best threshold
print("Best threshold:", best_threshold)

In [ ]:
threshold=best_threshold

In [ ]:
data['predicted_rainfall_class'] = np.where(data['mse_loss'] > threshold, 'extreme', 'normal')
data[data["mse_loss"]>threshold]

In [ ]:
wrong_predictions = data[data['predicted_rainfall_class'] != data['rainfall_class']]
percentage_of_error=(wrong_predictions.shape[0]/data.shape[0])*100

# Print the wrong predictions
print("wrong predictions = ",wrong_predictions.shape[0])
print("total data = ",data.shape[0])
print("percentage of error = ",percentage_of_error)

In [ ]:
num_correct = len(data[data['predicted_rainfall_class'] == data['rainfall_class']])
num_total = len(data)
accuracy = num_correct / num_total * 100
print('Accuracy: {:.2f}%'.format(accuracy))